In [3]:
import csv
import datetime
from time import time

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchviz import make_dot, make_dot_from_trace

import network

In [4]:
script_start = time()
print(f"Started: {datetime.datetime.now()}")

# continue training
cont = False

# %%
# Hardware acceleration
## accuracy vs epoch recording
epoch_accuracy_pair = []

#%%
## Hardware acceleration

torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# normalise the data
transform = transforms.Compose(
    [transforms.Resize(64),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# %%
# load data (download data from UToronto)
training_data = CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

test_data = CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

# load the dataset, describe the shape
# %%

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# image is of 32x32 with 3 channel for colours
# batch size is 64, but can be modified

for X, y in test_dataloader:

    print("Shape of X [n, Channels, Height, Width]: ", X.shape, X.dtype)
    print("Shape of y: ", y.shape, y.dtype)  # classification
    break

# %%
# Visualisation

# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze().permute(1,2,0)
# label = train_labels[0]
# plt.imshow(img)
# plt.show()
# print(f"Label: {label}")

# %%

network_model = network.Network()

# continue training -> load previous model
if cont:
    print("continuing previous progress.")
    network_model.load_state_dict(torch.load("model.pth"))
    network_model.eval()

network_model.to(device)  # send tensors to CUDA cores
print(network_model)

# logits = network_model(train_dataloader)
# pred_probab = nn.Softmax(dim=1)(logits)
# define hyper-parameters
batch_size = 64
# training

Started: 2021-05-09 21:36:44.720103
Files already downloaded and verified
Files already downloaded and verified
Shape of X [n, Channels, Height, Width]:  torch.Size([64, 3, 64, 64]) torch.float32
Shape of y:  torch.Size([64]) torch.int64
Network(
  (cnn_relu_stack): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): LeakyReLU(negative_slope=0.01, inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout2d(p=0.16, inplace=False)
    (7): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1))
    (8): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.01, inplace=True)
    (10): Dropout2d(p=0.16, inplace=False)
    (11): Conv2d(192, 384, kerne

In [12]:
def draw(dataloader, model: nn.Module):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # send data to device
        # predict
        pred = model(X)
        visgraph = make_dot(pred.mean(), params=dict(model.named_parameters()))
        visgraph.view()
        print("drawing completed")
        break

epochs = 50
max_accuracy = 0
consecutive = 0
max_consecutive = 50

draw(train_dataloader, network_model)

CalledProcessError: Command '['dot', '-Kdot', '-Tpdf', '-O', 'Digraph.gv']' died with <Signals.SIGKILL: 9>. [stderr: b'']